In [1]:
from focus_stack.stack_framework import *

In [2]:
job = StackJob("job", "E:/Focus stacking/2024-03-17 - Baltic Diptera/A/")
#job.add_action(AlignLayers(job.working_directory, "align", "Immagini modificate", method=AlignLayers.ALIGN_RIGID))
#job.add_action(BalanceLayers(job.working_directory, "balance", "align", method=BalanceLayers.BALANCE_LUMI, mask_radius=0.8, i_min=10, i_max=255))
#job.add_action(FocusStackBunch(job.working_directory, "batches", "balance", exif_dir="Immagini modificate", postfix='_stack_pyr', denoise=0.8, method=FocusStackBunch.METHOD_PYRAMID, energy=FocusStackBunch.ENERGY_LAPLACIAN))
job.add_action(FocusStack(job.working_directory, "stack", "batches", exif_dir="Immagini modificate", postfix='_stack_pyr', denoise=0.8, method=FocusStack.METHOD_PYRAMID, energy=FocusStack.ENERGY_LAPLACIAN))

In [3]:
job.run()

running stack
18 files in folder: 'E:/Focus stacking/2024-03-17 - Baltic Diptera/A/batches'
stack: elapsed time: 00:00:39.29s                    
stack completed                    
job: elapsed time: 00:00:39.29s                    
job completed                    
